In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import json
import folium
from pyproj import Proj, transform

import warnings
warnings.filterwarnings(action='ignore')

from gurobipy import *
import gurobipy as gp

## 모델링 준비 - 변경 불필요 Sell
* 모델링을 준비하기 위한 사전 작업
* 기본적으로 고정된 변수이기에, 아래 셀의 내용은 바꾸지 않으며 해당 엑셀도 건드리지 않는다.

In [2]:
# 이웃노드에 대한 정의, 그리드 자체에 변경이 생기지 않는 한 해당 데이터는 바꾸지 않는다.
neighbor_node = pd.read_excel('neighbor_node.xlsx').iloc[:,1:].fillna(0)

# 이웃노드간 이동 경로를 담고 있는 리스트
move_list = [(i,j) for i in range(84) for j in range(84) if neighbor_node.iloc[i,j]==1]

# 그리드의 index와 해당 그리드의 가공 전 index 정보를 가져옴으로써 향후 그리드 데이터를 가공할 수 있게 함
grid_data = pd.read_excel('grid_data.xlsx')

# 그리드 확인을 위해 좌표계를 담고 있는 변수 선언
proj_2097 = Proj(init='EPSG:2097')
proj_4326 = Proj(init='EPSG:4326')

## 모델링 준비 - 변경 가능 Sell
* 모델링을 진행하기에 앞서, 데이터가 변경된 점이 있다면 해당 Sell에서 호출하는 엑셀 데이터를 바꿔 진행한다.
* 이 때, 기존과 같은 형식에 값만 바꿈으로써 형태가 달라져 프로그램이 작동하지 않는 경우가 없게 한다.

In [3]:
# 각 그리드별 수요를 담고 있는 Data, 차후 수요에 대한 변경이 있을 시 해당 데이터를 바꿔주면 됨
demand = pd.read_excel('오토바이 등록대수_그리드.xlsx')

# 이미 설치되어있는 기존 충전소의 위치를 나타냄. 이 때, 다른 Column은 제외하더라도 '위도', '경도' 만은 정확히 입력함
zentropy = pd.read_excel('zentropy.xlsx')

# 새로 충전소를 설치하고자 하는 후보지. 이 때, 다른 Column은 제외하더라도 '위도', '경도' 만은 정확히 입력함
candidate_site = pd.read_excel('convenienceStore.xlsx')

# 이동 경로에 따른 수요를 담고 있는 리스트
demand_list = [(demand.iloc[i,0]+demand.iloc[j,0])/2 for i,j in move_list]

# 경쟁사 충전소 위치 데이터
competitor = pd.read_csv('dna_motors.csv', encoding='euc_kr')

### 함수 정의 - 경로 탐색 함수
#### 함수의 input: (이웃노드, 이웃노드로 이동하는 경로, 그리드 데이터, 충전소 후보지, 기존 충전소)
* 충전소 후보지 중에 서울을 벗어나는 후보지가 있는지 탐색한다.
* 서울 내 충전소 후보지 확인을 완료한 후, 노드 -> 노드로 이동할 때 방문 가능한 이웃 충전소를 탐색하는 과정을 거친다.
* 탐색 후 이동 가능한 모든 경로를 Return한다.
* 이와 같은 함수를 통해 충전소를 방문하는 모든 경로를 알 수 있으며, 이를 바탕으로 로드밸런싱을 진행할 수 있다.

In [4]:
def make_path(neighbor_node_input, move_list_input, grid_data_input, candidate_site_input, zentropy_input):
    
    charging_station = pd.concat([candidate_site_input[['위도', '경도']], zentropy_input[['위도', '경도']]])
    charging_station['index_candidate_zentropy'] = charging_station.index
    charging_station = charging_station.reset_index(drop=True)
    charging_station['is_constructed'] = np.where(charging_station.index < len(candidate_site_input), 0, 1)
    
    charging_station_latlon = transform(proj_4326, proj_2097, charging_station['경도'], charging_station['위도'])
    charging_station_latlon = pd.DataFrame(sorted([charging_station_latlon[0].tolist(), charging_station_latlon[1].tolist()]), 
                                         index=('경도', '위도')).transpose()
    # 위도, 경도에 따른 그리드 확인 후 후보지 위경도가 서울을 벗어나는지 여부 확인
    charging_station_latlon['x_grid'] = (charging_station_latlon['경도'] -180000)//3000
    charging_station_latlon['y_grid'] = (charging_station_latlon['위도'] -435000)//3000
    charging_station_latlon['x_error'] = np.where((charging_station_latlon['x_grid']>= 0) 
                                                & (charging_station_latlon['x_grid'] <= 13), "No", "Yes")
    charging_station_latlon['y_error'] = np.where((charging_station_latlon['y_grid']>= 0) 
                                                & (charging_station_latlon['y_grid'] <= 11), "No", "Yes")
    charging_station_latlon['error'] = np.where((charging_station_latlon['x_error'] == 'No') 
                                              & (charging_station_latlon['y_error'] == 'No'), "No", "Yes")
    charging_station_latlon['grid'] = charging_station_latlon['x_grid']*12 + charging_station_latlon['y_grid']
    
    # 그리드 인덱스 값 저장
    charging_station['index_before'] = charging_station_latlon['grid']

    # 위경도가 서울시를 벗어나는 경우 필터링
    charging_station_filtered = charging_station[charging_station_latlon['error'] == 'No']
    charging_station_filtered['index_before'] = charging_station_filtered['index_before'].astype(int)

    # 서울시를 벗어나는 경우 필터링한 후 충전소 후보지 별 그리드 위치 저장
    charging_station_filtered = pd.merge(charging_station_filtered, grid_data_input, on='index_before', how='left')
    charging_station_filtered = charging_station_filtered.dropna()

    # 그리드 인덱스를 저장하는 데이터프레임 생성
    charging_station_index = charging_station_filtered[['index']].reset_index(drop=True)
    charging_station_index['index'] = charging_station_index['index'].astype(int)
    
    # 기점노드에서 종점노드로 이동할 때, 모든 충전소 후보지를 방문하는 경로를 담고 있는 리스트 생성
    charging_station_list = [1 if ((neighbor_node_input[i][charging_station_index['index'][k]]==1.0)
                                 |(neighbor_node_input[j][charging_station_index['index'][k]]==1.0)) 
                           else 0 for i,j in move_list_input for k in range(len(charging_station_index))]

    # 충전소 후보지는 이웃노드로만 갈 수 있기에, 모든 충전소 후보지를 방문하는 경우에서 이웃노드 방문하는 경우만 추려냄
    station_visit_list = [(i,j,k) for i,j in move_list_input for k in range(len(charging_station_index)) 
                          if charging_station_list[len(charging_station_index)*move_list_input.index((i,j))+k] == 1]
    
    return [charging_station_filtered, station_visit_list]

### 함수 실행
* 함수 실행 결과 얻을 수 있는 필터링된 충전소 데이터, 충전소 방문 경로 저장

In [80]:
make_path_res = make_path(neighbor_node, move_list, grid_data, candidate_site, zentropy)

station_filtered = make_path_res[0]
station_visit_list = make_path_res[1]

### 함수 정의 - Gurobi Solve 함수
#### 함수의 input: (충전소 후보지&기존 충전소, 충전소 방문하여 이동하는 경로, 이웃노드로 이동하는 경로, 수요 정보, MIP 시간제한 (초), 충전소 대수 (기존 충전소 제외))

* i노드에서 j노드로 갈 때, 반드시 하나의 충전소를 방문한다.
* 충전소 후보지 k에 충전소가 건설이 안 된다면, 해당 충전소를 방문할 수 없다.
* 충전소 부하 최대값인 L은 k 충전소의 부하보다 크다.
* 충전소 후보지 k에 건설되는 충전소의 총합은 N개이다.
* 기존에 설치되어 있는 충전소는 반드시 건설된다.

In [5]:
def solve_mip(station_input, station_visit_list_input, move_list_input, demand_list_input, time_limit, station_number):
    station_visit_df = pd.DataFrame(station_visit_list_input)
    
    m = Model('LoadBalancing')
    # m.ModelSense = GRB.MINIMIZE

    # Guribi의 Time Limit 설정
    m.setParam('TimeLimit', time_limit)

    # 충전소 후보지 충전소 건설 여부
    x = m.addVars(len(station_input), vtype=GRB.BINARY, name="x")
    # i노드에서 j노드로 이동할 때 k충전소 방문하는지 여부
    y = m.addVars(station_visit_list_input, vtype=GRB.BINARY, name="y")
    # 충전소 부하 최대치
    L = m.addVar(vtype=GRB.CONTINUOUS, name="L_max")



    # 충전소 최대 부하값을 최소화
    m.setObjective(L, GRB.MINIMIZE)

    # i노드에서 j노드로 이동할 때 반드시 하나의 충전소를 방문
    m.addConstrs((y.sum(i,j,'*') == 1 for i,j in move_list_input), 'must_charger_visit')
    # 충전소 후보지 k에 충전소가 설치되어 있지 않으면 방문하지 않음
    m.addConstrs((y[i,j,k] <= x[k] for i,j,k in station_visit_list_input), 'no_charger_no_visit')
    # 충전소 후보지 k의 부하 총합보다 L이 커야 함
    for k in range(len(station_input)):
        m.addConstr((gp.quicksum(demand_list_input[move_list_input.index((i,j))]*y[i,j,k] 
                                 for i,j in station_visit_df[station_visit_df[2] == k].iloc[:,:-1].values.tolist())) <= L )
    # m.addConstrs((y.sum('*','*',k) <= L for k in range(len(conv_index))), 'L_max meaning')
    # m.addConstrs((x.sum(k) <= 100 for k in range(len(conv_index))), 'charger_limit')
    # 충전소의 총합은 n개를 넘어가지 않음
    m.addConstr((gp.quicksum(x[k] for k in range(len(station_input))) <= station_number + station_input['is_constructed'].sum()), 
                'charger_limit')
    # 기존에 설치되어 있는 충전소는 반드시 설치되어야 됨
    m.addConstrs((x[k] == 1 for k in range(len(station_input)-station_input['is_constructed'].sum(), len(station_input))),
                 'existing_charger')

    m.optimize()
    
    return [v.X for v in m.getVars() if v.VarName[0]=="x"]

### 함수 실행
* 300초의 시간 제한으로 40대를 설치하며 결과를 확인

In [127]:
res = solve_mip(station_filtered, station_visit_list, move_list, demand_list, 300, 40)

Set parameter TimeLimit to value 300
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: Intel(R) Core(TM) i5-8265U CPU @ 1.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 98895 rows, 98154 columns and 390628 nonzeros
Model fingerprint: 0x30e8ed02
Variable types: 1 continuous, 98153 integer (98153 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+05]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]
Presolve removed 10960 rows and 110 columns
Presolve time: 1.43s
Presolved: 87935 rows, 98044 columns, 368708 nonzeros
Variable types: 1 continuous, 98043 integer (98043 binary)
Found heuristic solution: objective 1.385490e+07
Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...

Concurrent spin time: 0.02s

Solved with primal simplex

Use crossover to convert LP symmetric solution to ba

### 충전소 건설 여부 확인

In [128]:
station_filtered['mip_result'] = res
station_filtered

,위도,경도,index_candidate_zentropy,is_constructed,index_before,index,mip_result
0,37.499411,126.998426,0,0,75,38.0,1.0
1,37.480639,126.960935,2,0,62,29.0,0.0
2,37.573624,127.025082,3,0,89,49.0,0.0
3,37.558571,127.151149,4,0,137,82.0,0.0
4,37.546207,127.120526,5,0,124,77.0,0.0
...,...,...,...,...,...,...,...
1044,37.550024,127.144832,105,1,124,77.0,1.0
1045,37.540845,127.146409,106,1,136,81.0,1.0
1046,37.561334,127.169714,107,1,137,82.0,1.0
1047,37.555454,127.136241,108,1,125,78.0,1.0


## MIP 결과는 '그리드 간 배치' 이기에, 그리드 내 충전소를 배치하는 다른 알고리즘이 필요하다. 이를 위해, 두 가지 알고리즘을 이용하여 그리드 내 충전소 배치를 진행한다.
### 알고리즘 1
* 경쟁사의 충전소 거리를 비교하여, 경쟁사 충전소와 가장 먼 곳에 위치한 후보지에 충전소를 설치한다. 이 때, 새로 설치되는 충전소 간 거리는 고려하지 않는다.

In [6]:
def construct_by_1st_algorithm(station_input, grid_data_input, competitor_input):
    candidate = station_input[station_input['is_constructed'] == 0]
    
    dist=[]
    for i in candidate.index:
        minimum = 100
        for j in competitor_input.index:
            if (abs(candidate['위도'][i]-competitor_input['위도'][j])+
                abs(candidate['경도'][i]-competitor_input['경도'][j])) < minimum:
                minimum = (abs(candidate['위도'][i]-competitor_input['위도'][j])+
                           abs(candidate['경도'][i]-competitor_input['경도'][j]))
        dist.append(minimum)
    candidate['minimum_dist'] = dist
    
    grid_data_input['station'] = 0
    
    candidate_construct = candidate[candidate['mip_result'] == 1].reset_index(drop=True)
    
    for i in range(len(candidate_construct)):
        grid_data_input['station'][candidate_construct['index'][i]] += 1
        
    candidate['rank'] = candidate.groupby('index')['minimum_dist'].rank(method='min', ascending=False)
    candidate = candidate.sort_values(by=['index', 'rank']).reset_index(drop=True)
    candidate = pd.merge(candidate, grid_data_input[['index', 'station']], on='index', how='left')
    candidate['mip_result'] = np.where(candidate['rank'] <= candidate['station'], 1, 0)
    
    return candidate

### 알고리즘 2
* 경쟁사의 충전소 거리를 비교하여 그리드 내 첫 충전소를 설치한다. 이후, 다음으로 설치되는 충전소는 직전에 설치한 충전소와 가장 거리가 먼 후보지에 설치한다.

In [7]:
def construct_by_2nd_algorithm(station_input, grid_data_input, competitor_input):
    candidate = station_input[station_input['is_constructed'] == 0]
    
    dist=[]
    for i in candidate.index:
        minimum = 100
        for j in competitor_input.index:
            if (abs(candidate['위도'][i]-competitor_input['위도'][j])+
                abs(candidate['경도'][i]-competitor_input['경도'][j])) < minimum:
                minimum = (abs(candidate['위도'][i]-competitor_input['위도'][j])+
                           abs(candidate['경도'][i]-competitor_input['경도'][j]))
        dist.append(minimum)
    candidate['minimum_dist'] = dist
    
    grid_data_input['station'] = 0
    
    candidate_construct = candidate[candidate['mip_result'] == 1].reset_index(drop=True)
    
    for i in range(len(candidate_construct)):
        grid_data_input['station'][candidate_construct['index'][i]] += 1
        
    candidate['rank'] = candidate.groupby('index')['minimum_dist'].rank(method='min', ascending=False)
    candidate = candidate.sort_values(by=['index', 'rank']).reset_index(drop=True)
    candidate = pd.merge(candidate, grid_data_input[['index', 'station']], on='index', how='left')
    candidate['mip_result'] = 0

    for i in range(len(grid_data_input)):
        grid_df = candidate[candidate['index'] == i]
        if len(grid_df) == 0:
            continue
        station_grid = grid_df['station'].values[0]
        if station_grid == 0:
            continue
        if station_grid == 1:
            candidate['mip_result'][grid_df.index[0]] = 1
            continue
        grid_df['mip_result'][grid_df.index[0]] = 1
        latlon_standard = grid_df[['위도','경도']].iloc[0]
        for j in range(1, station_grid):
            grid_df['dist'] = abs(latlon_standard[0] - grid_df['위도']) + abs(latlon_standard[1] - grid_df['경도'])
            max_idx = grid_df[grid_df['mip_result'] == 0]['dist'].idxmax()
            grid_df['mip_result'][max_idx] = 1
            latlon_standard = candidate[['위도','경도']].iloc[max_idx]
        candidate.at[grid_df[grid_df['mip_result'] == 1].index, 'mip_result']=1

    
    return candidate

In [131]:
algorithm_1st = construct_by_1st_algorithm(station_filtered, grid_data, competitor)

In [132]:
algorithm_2nd = construct_by_2nd_algorithm(station_filtered, grid_data, competitor)

In [134]:
algorithm_1st

,위도,경도,index_candidate_zentropy,is_constructed,index_before,index,mip_result,minimum_dist,rank,station
0,37.483027,126.821508,623,0,14,3.0,0,0.010233,1.0,0
1,37.480800,126.824325,102,0,14,3.0,0,0.009642,2.0,0
2,37.487138,126.831167,560,0,14,3.0,0,0.008161,3.0,0
3,37.487748,126.824396,77,0,14,3.0,0,0.002624,4.0,0
4,37.513979,126.834415,205,0,15,4.0,0,0.024272,1.0,0
...,...,...,...,...,...,...,...,...,...,...
932,37.554524,127.154236,465,0,137,82.0,0,0.037259,36.0,0
933,37.553958,127.154637,605,0,137,82.0,0,0.037094,37.0,0
934,37.553063,127.155504,410,0,137,82.0,0,0.037066,38.0,0
935,37.559555,127.179251,364,0,149,83.0,0,0.067306,1.0,0


### 선정된 후보지를 지도에 Plotting하는 함수 작성

In [8]:
def plotting(candidate_input):
    m = folium.Map(location=[37.5642135, 127.0016985], zoom_start=11)

    for _, r in candidate_input[candidate_input['mip_result'] == 1].iterrows():
        location = (r['위도'], r['경도']) # 위도, 경도 튜플
        folium.Circle(
            location=location,
            color='red',
            weight=5,
            fill_opacity=1,
            opacity=5,
            fill_color='red',
            fill=True,  # gets overridden by fill_color
            # popup=str(row['Id'])
        ).add_to(m)

    return m

def plotting_all(candidate_input, zentropy_input):
    m = folium.Map(location=[37.5642135, 127.0016985], zoom_start=11)

    for _, r in candidate_input[candidate_input['mip_result'] == 1].iterrows():
        location = (r['위도'], r['경도']) # 위도, 경도 튜플
        folium.Circle(
            location=location,
            color='red',
            weight=5,
            fill_opacity=1,
            opacity=5,
            fill_color='red',
            fill=True,  # gets overridden by fill_color
            # popup=str(row['Id'])
        ).add_to(m)
        
    for _, r in zentropy_input.iterrows():
        location = (r['위도'], r['경도']) # 위도, 경도 튜플
        folium.Circle(
            location=location,
            color='blue',
            weight=5,
            fill_opacity=1,
            opacity=5,
            fill_color='blue',
            fill=True,  # gets overridden by fill_color
            # popup=str(row['Id'])
        ).add_to(m)

    return m

In [140]:
plotting(algorithm_1st)

In [141]:
plotting_all(algorithm_1st, zentropy)